<a href="https://colab.research.google.com/github/LoreJob/DeepFake-Dct/blob/main/CNN+LSTM Third approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN+LSTM for DeepFake Detection in videos


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Ram/Disk on the upper-left part of the colab → Additional Connection Options
- select GPU T4 from the Hardware Accelerator drop-down (You have just 1 hour of using)

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [29]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

## Packages
As the title says, we are using the tensorflow package.

In [3]:
import os
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.utils import Sequence
from tqdm import tqdm

### Variables

In [4]:
VIDEO_WIDTH, VIDEO_HEIGHT = 128, 128
MAX_FRAMES = 20  # Number of frames of a video that will be fed to the model as one sequence
DATASET_DIR = "Video Dataset Small"
PROCESSED_DIR = "D:\\Users\\megam\\Artificial Project\\Processed_data_128x128"
CLASSES = ["fake", "real"]
BATCH_SIZE = 32

## Importing the dataset using data generators

This method is not loading all the data, but is creating some data generators that are extracting data when it's needed, with a batch for batch approach.
This methos is light on the RAM but slow when you run the model.

### Preprocessing data

To speed up the model, we preprocessed data, in order to speed up training by reducing the time spent on loading and processing videos during each epoch.

YOU DON'T NEED TO RUN THIS CELL unless your "Video Dataset Small/Processed_data" folder is empty or you want to change Variables

In [9]:
# def preprocess_video(video_path):
#     cap = cv2.VideoCapture(video_path)
#     total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
#     if total_frames == 0:
#         return np.zeros((MAX_FRAMES, VIDEO_HEIGHT, VIDEO_WIDTH, 3))

#     frames_to_sample = min(MAX_FRAMES, total_frames)
#     frame_indices = np.linspace(0, total_frames - 1, frames_to_sample, dtype=int)
    
#     frames = []
#     for frame_index in frame_indices:
#         cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
#         ret, frame = cap.read()
#         if ret:
#             frame = cv2.resize(frame, (VIDEO_WIDTH, VIDEO_HEIGHT))
#             frame = frame / 255.0  # Normalize pixel values
#             frames.append(frame)
#         else:
#             frames.append(np.zeros((VIDEO_HEIGHT, VIDEO_WIDTH, 3)))
    
#     cap.release()

#     # Pad with zeros if we couldn't extract enough frames
#     if len(frames) < MAX_FRAMES:
#         padding = [np.zeros((VIDEO_HEIGHT, VIDEO_WIDTH, 3)) for _ in range(MAX_FRAMES - len(frames))]
#         frames.extend(padding)

#     return np.array(frames[:MAX_FRAMES])

# def process_dataset():
#     for subset in ['Train', 'Val', 'Test']:
#         for class_name in ['real', 'fake']:
#             input_dir = os.path.join(DATASET_DIR, subset, class_name)
#             output_dir = os.path.join(PROCESSED_DIR, subset, class_name)
#             os.makedirs(output_dir, exist_ok=True)
            
#             video_files = [f for f in os.listdir(input_dir) if f.endswith('.mp4')]
            
#             for video_file in tqdm(video_files, desc=f"Processing {subset} {class_name}"):
#                 video_path = os.path.join(input_dir, video_file)
#                 processed_frames = preprocess_video(video_path)
                
#                 output_path = os.path.join(output_dir, video_file.replace('.mp4', '.npy'))
#                 np.save(output_path, processed_frames)

# if __name__ == "__main__":
#     process_dataset()

Processing Test fake: 100%|██████████| 1500/1500 [03:32<00:00,  7.07it/s]


Running time for the previous cell on my pc was one hour. Colab is slower -Manu

In [5]:
class VideoDataGenerator(Sequence):
    def __init__(self, data_dir, subset, batch_size=BATCH_SIZE):
        self.data_dir = data_dir
        self.subset = subset
        self.batch_size = batch_size
        self.classes = ['real', 'fake']
        self.videos = self._get_video_paths()
        self.on_epoch_end()

    def _get_video_paths(self):
        videos = []
        subset_dir = os.path.join(self.data_dir, self.subset)
        for class_name in self.classes:
            class_dir = os.path.join(subset_dir, class_name)
            for video_name in os.listdir(class_dir):
                videos.append((os.path.join(class_dir, video_name), self.classes.index(class_name)))
        return videos

    def __len__(self):
        return len(self.videos) // self.batch_size

    def __getitem__(self, idx):
        batch_videos = self.videos[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_frames = []
        batch_labels = []

        for video_path, label in batch_videos:
            frames = np.load(video_path)
            batch_frames.append(frames)
            batch_labels.append(label)

        return np.array(batch_frames), np.array(batch_labels)

    def on_epoch_end(self):
        np.random.shuffle(self.videos)


In [6]:
train_generator = VideoDataGenerator(PROCESSED_DIR, "Train")
val_generator = VideoDataGenerator(PROCESSED_DIR, "Val")
test_generator = VideoDataGenerator(PROCESSED_DIR, "Test")

## Training the model

### Building the CNN model

In [16]:
# # Here we will add the model that performed the best on the img dataset without the last dense layer

# CNN_model = models.Sequential([
#     layers.Conv2D(16, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     layers.MaxPooling2D(),
#     layers.Dropout(0.1),  # Dropout layer with 10% rate

#     layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     layers.MaxPooling2D((2,2)),
#     layers.Dropout(0.1),  # Dropout layer with 10% rate

#     layers.Conv2D(64, (5, 5), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     layers.MaxPooling2D((2,2)),
#     layers.Dropout(0.1),  # Dropout layer with 10% rate

#     layers.Flatten(),

#     layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),

# ])

In [26]:
from tensorflow.keras.models import load_model
InceptionV3 = load_model('Epoch_lore\\model_epoch_10.h5')

InceptionV3.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,422,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,811,203 (25.98 MB)

 Trainable params: 6,811,201 (25.98 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [27]:
new_model = tf.keras.Sequential()

# Add an input layer with the appropriate shape
input_shape = InceptionV3.input_shape[1:]  # Remove the batch dimension
new_model.add(tf.keras.layers.InputLayer(input_shape=input_shape))

# Add layers from the original model, except the last two layers
for layer in InceptionV3.layers[:-2]:
    new_model.add(layer)

# Make sure the layers are not recompiled
for layer in new_model.layers:
    layer.trainable = False

new_model.summary()

c:\Users\megam\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,422,656 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,811,072 (25.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 6,811,072 (25.98 MB)

Il modello caricato è quello semplice fatto da all'inizio da lore.

### Creating the full model

Quì si rompe perchè il modello era trainato su immagini 256x256.
Ho provato a fare un preprocess su 200 immagini con dimensioni 256*256 e il codice funziona. Quindi caricando un modello che prende come input immagini della grandezza giusta dovrebbe funzionare.

I modelli già pronti (tipo inception V3) non funzionano. Ho perso 3 pomeriggi per provare a farlo funzionare ma niente da fare, il modello bisogna che venga creato from scratch. -Manu

In [28]:
CNN_LSTM_model = models.Sequential()
# Input
CNN_LSTM_model.add(layers.Input((MAX_FRAMES, VIDEO_WIDTH, VIDEO_HEIGHT, 3))) # 3 are the channels
# Adding the time distributed CNN
CNN_LSTM_model.add(layers.TimeDistributed(new_model)) 
# Creating the LSTM part
CNN_LSTM_model.add(layers.LSTM(64, return_sequences=False))
CNN_LSTM_model.add(layers.Dense(64, activation='relu'))
CNN_LSTM_model.add(layers.Dropout(0.25)) # Dropout layer with 25% rate
CNN_LSTM_model.add(layers.Dense(1, activation='sigmoid'))

# Compiling the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

CNN_LSTM_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

CNN_LSTM_model.summary()

ValueError: Exception encountered when calling TimeDistributed.call().

[1mInput 0 of layer "functional_187" is incompatible with the layer: expected shape=(None, 256, 256, 3), found shape=(None, 128, 128, 3)[0m

Arguments received by TimeDistributed.call():
  • args=('<KerasTensor shape=(None, 20, 128, 128, 3), dtype=float32, sparse=None, name=keras_tensor_2523>',)
  • kwargs={'mask': 'None'}

## Training the model

In [29]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_epoch_{epoch:02d}.keras',
    save_best_only=False,
    save_weights_only=False,
    save_freq='epoch')

history = CNN_LSTM_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[checkpoint_callback])

ValueError: You must call `compile()` before using the model.